In [ ]:
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
import keras
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import KFold

my_images = []
labels = [0] * 5 + [1] * 10

for i in range(15):
    file = f"./my_images/img{i + 1:02d}.jpg"
    image = cv.imread(file)
    image = cv.resize(image, (96, 96))
    image = cv.cvtColor(image, cv.COLOR_BGR2RGB)
    my_images.append(image)

X = np.array(my_images, dtype='float32') / 255.0
y = np.array(labels)


def mixup(x, y, alpha=0.2):
    lam = np.random.beta(alpha, alpha)
    index = np.random.permutation(len(x))
    mixed_x = lam * x + (1 - lam) * x[index]
    mixed_y = lam * y + (1 - lam) * y[index]
    return mixed_x, mixed_y


def cutmix(x, y, alpha=1.0):
    lam = np.random.beta(alpha, alpha)
    index = np.random.permutation(len(x))
    h, w = x.shape[1], x.shape[2]
    cx = np.random.randint(w)
    cy = np.random.randint(h)
    cut_w = int(w * np.sqrt(1 - lam))
    cut_h = int(h * np.sqrt(1 - lam))
    x1 = np.clip(cx - cut_w // 2, 0, w)
    y1 = np.clip(cy - cut_h // 2, 0, h)
    x2 = np.clip(cx + cut_w // 2, 0, w)
    y2 = np.clip(cy + cut_h // 2, 0, h)
    x_cutmix = x.copy()
    x_cutmix[:, y1:y2, x1:x2, :] = x[index, y1:y2, x1:x2, :]
    lam_adjusted = 1 - ((x2 - x1) * (y2 - y1) / (w * h))
    y_cutmix = lam_adjusted * y + (1 - lam_adjusted) * y[index]
    return x_cutmix, y_cutmix


datagen = ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.25,
    height_shift_range=0.25,
    zoom_range=0.4,
    horizontal_flip=True,
    brightness_range=[0.6, 1.4],
    fill_mode='nearest'
)

kf = KFold(n_splits=5, shuffle=True, random_state=42)
models = []

for fold, (train_index, val_index) in enumerate(kf.split(X), 1):
    print(f"\n--- Fold {fold} ---")
    X_train, X_val = X[train_index], X[val_index]
    y_train, y_val = y[train_index], y[val_index]

    datagen.fit(X_train)

    # MixUp 적용
    X_train_mix, y_train_mix = mixup(X_train, y_train)

    # CutMix 적용
    X_train_mix, y_train_mix = cutmix(X_train_mix, y_train_mix)

    # CNN 모델 정의
    model = keras.Sequential([
        keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(96, 96, 3)),
        keras.layers.BatchNormalization(),
        keras.layers.MaxPooling2D((2, 2)),

        keras.layers.Conv2D(64, (3, 3), activation='relu'),
        keras.layers.BatchNormalization(),
        keras.layers.MaxPooling2D((2, 2)),

        keras.layers.Conv2D(64, (3, 3), activation='relu'),
        keras.layers.BatchNormalization(),
        keras.layers.Flatten(),

        keras.layers.Dense(128, activation='relu'),
        keras.layers.Dropout(0.5),
        keras.layers.Dense(64, activation='relu'),
        keras.layers.Dropout(0.3),
        keras.layers.Dense(1, activation='sigmoid')
    ])

    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
        loss='binary_crossentropy',
        metrics=['accuracy']
    )

    early_stop = EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)

    history = model.fit(
        datagen.flow(X_train_mix, y_train_mix, batch_size=2),
        validation_data=(X_val, y_val),
        epochs=200,
        callbacks=[early_stop],
        verbose=1
    )

    models.append(model)

test_images = []
for i in range(10):
    file = f"./test_images/img{i + 1:02d}.jpg"
    image = cv.imread(file)
    image = cv.resize(image, (96, 96))
    image = cv.cvtColor(image, cv.COLOR_BGR2RGB)
    test_images.append(image)

X_test = np.array(test_images, dtype='float32') / 255.0

predictions = np.zeros((len(X_test), 1))
for model in models:
    predictions += model.predict(X_test)
predictions /= len(models)

for i, p in enumerate(predictions):
    if p > 0.5:
        print(f"img{i + 1:02d}.jpg → ✅ 내 얼굴 ({p[0]:.3f})")
    else:
        print(f"img{i + 1:02d}.jpg → ❌ 타인 ({p[0]:.3f})")



--- Fold 1 ---
Epoch 1/200
6/6 ━━━━━━━━━━━━━━━━━━━━ 3s 83ms/step - accuracy: 0.0000e+00 - loss: 0.9870 - val_accuracy: 0.3333 - val_loss: 0.7470
Epoch 2/200
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step - accuracy: 0.0000e+00 - loss: 0.6871 - val_accuracy: 0.3333 - val_loss: 0.8591
Epoch 3/200
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step - accuracy: 0.0000e+00 - loss: 0.6925 - val_accuracy: 0.3333 - val_loss: 0.9750
Epoch 4/200
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step - accuracy: 0.0000e+00 - loss: 0.6920 - val_accuracy: 0.3333 - val_loss: 1.0840
Epoch 5/200
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step - accuracy: 0.0000e+00 - loss: 0.6686 - val_accuracy: 0.3333 - val_loss: 1.2312
Epoch 6/200
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - accuracy: 0.0000e+00 - loss: 0.9301 - val_accuracy: 0.3333 - val_loss: 1.4452
Epoch 7/200
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - accuracy: 0.0000e+00 - loss: 0.5825 - val_accuracy: 0.3333 - val_loss: 1.5770
Epoch 8/200
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - accuracy: 0.0833 - los